In [ ]:
from collections import deque

# --- Read Graph from File ---
def read_graph_from_file(filename):
    graph = {}
    source, dest = None, None
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("Source:"):
                source = line.split(":")[1].strip()
            elif line.startswith("Destination:"):
                dest = line.split(":")[1].strip()
            else:
                parts = line.split()
                node = parts[0]
                neighbors = parts[1:] if len(parts) > 1 else []
                graph[node] = neighbors
                # make sure neighbors bhi graph me aa jaye
                for nb in neighbors:
                    if nb not in graph:
                        graph[nb] = []
    return graph, source, dest


# --- Reconstruct Path from parent dictionary ---
def reconstruct_path(parent, source, dest):
    path = []
    curr = dest
    while curr is not None:
        path.append(curr)
        curr = parent.get(curr, None)
    path.reverse()
    if path and path[0] == source:
        return path
    else:
        return None


# --- BFS for shortest path ---
def bfs_shortest_path(graph, source, dest):
    color = {u: "white" for u in graph}
    distance = {u: float("inf") for u in graph}
    parent = {u: None for u in graph}

    queue = deque([source])
    color[source] = "grey"
    distance[source] = 0

    while queue:
        u = queue.popleft()
        for v in graph[u]:
            if color[v] == "white":
                color[v] = "grey"
                distance[v] = distance[u] + 1
                parent[v] = u
                queue.append(v)
        color[u] = "black"

    return distance, parent


# --- DFS Helper ---
def dfs_visit(graph, u, color, parent, discovery, finish, time):
    color[u] = "grey"
    time[0] += 1
    discovery[u] = time[0]
    for v in graph[u]:
        if color[v] == "white":
            parent[v] = u
            dfs_visit(graph, v, color, parent, discovery, finish, time)
    color[u] = "black"
    time[0] += 1
    finish[u] = time[0]


# --- DFS (to find *a* path) ---
def dfs_shortest_path(graph, source, dest):
    color = {u: "white" for u in graph}
    parent = {u: None for u in graph}
    discovery, finish = {}, {}
    time = [0]

    dfs_visit(graph, source, color, parent, discovery, finish, time)
    path = reconstruct_path(parent, source, dest)

    # DFS doesn’t guarantee shortest path
    dist = len(path) - 1 if path else None
    return dist, parent, path


# --- Main ---
if __name__ == "__main__":
    graph, source, dest = read_graph_from_file("graph.txt")

    print("Graph:", graph)
    print("Source:", source, "Destination:", dest)

    # BFS shortest path
    bfs_distances, bfs_parent = bfs_shortest_path(graph, source, dest)
    bfs_path = reconstruct_path(bfs_parent, source, dest)
    bfs_dist = bfs_distances[dest] if bfs_path else None
    print("\nBFS Path:", bfs_path)
    print("BFS Shortest Distance:", bfs_dist)

    # DFS path
    dfs_dist, dfs_parent, dfs_path = dfs_shortest_path(graph, source, dest)
    print("\nDFS Path:", dfs_path)
    print("DFS Distance (not guaranteed shortest):", dfs_dist)

    # Comparison
    if bfs_dist is not None and dfs_dist is not None:
        if bfs_dist < dfs_dist:
            print("\n BFS is better (gives shortest path).")
        elif dfs_dist < bfs_dist:
            print("\n DFS gave a shorter path here (rare case).")
        else:
            print("\nBoth BFS and DFS gave the same distance.")
    else:
        print("\nNo path exists between Source and Destination.")


Graph: {'A': ['B', 'C'], 'B': ['D', 'E'], 'C': ['F'], 'D': [], 'E': ['F'], 'F': []}
Source: A Destination: F

BFS Path: ['A', 'C', 'F']
BFS Shortest Distance: 2

DFS Path: ['A', 'B', 'E', 'F']
DFS Distance (not guaranteed shortest): 3

✅ BFS is better (gives shortest path).
